### This notebook is broken down into 6 parts:

- Initialize Network.
- Forward Propagate.
- Back Propagate Error.
- Train Network.
- Predict.
- Seeds Dataset Case Study.

These steps will provide the foundation that you need to implement the backpropagation algorithm from scratch and apply it to your own predictive modeling problems.

Source : https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

### 1. Initialize Network and Data

In [1]:
# Initialization - Exo TD : 1 hidden layer (2 neurons) and 1 ontput layer (2 neurons)
network = [[{'weights': [0.15, 0.20, 0.35]}, {'weights': [0.25, 0.30, 0.35]}],
           [{'weights': [0.40, 0.45, 0.60]}, {'weights': [0.50, 0.55, 0.60]}]]
print(network)

[[{'weights': [0.15, 0.2, 0.35]}, {'weights': [0.25, 0.3, 0.35]}], [{'weights': [0.4, 0.45, 0.6]}, {'weights': [0.5, 0.55, 0.6]}]]


In [2]:
# data row and its class
row_data = [0.05, 0.10, 0.01]
expected = [0.01, 0.99]

### 2. Forward Propagation

We can break forward propagation down into three parts:
- Neuron Activation (= neuron input).
- Neuron Transfer (=neuron output).
- Forward Propagation.

In [3]:
# Calculate neuron activation for an input
# activation = sum(weight_i * input_i) + bias
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

In [4]:
# Transfer neuron activation
# output = 1 / (1 + e^(-activation))
from math import exp
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

In [5]:
# Forward propagate input to a network output
def forward_propagate(network, row_data):
    inputs = row_data
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            print("Inputs : " + str(activation))
            neuron['output'] = transfer(activation)
            print("Outputs : " + str(neuron['output']))
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [6]:
# Forward propagation on : exo TD
output = forward_propagate(network, row_data)

Inputs : 0.3775
Outputs : 0.5932699921071872
Inputs : 0.39249999999999996
Outputs : 0.596884378259767
Inputs : 1.1059059670597702
Outputs : 0.7513650695523157
Inputs : 1.2249214040964653
Outputs : 0.7729284653214625


### 3. Back Propagate Error

This part is broken down into two sections.
- Transfer Derivative.
- Error Backpropagation.

In [7]:
# Calculate the derivative of an neuron output
# derivative = output * (1.0 - output)
def transfer_derivative(output):
    return output * (1.0 - output)

In [8]:
# Backpropagate error and store in neurons
# output_error = (expected - output) * transfer_derivative(output)
# hidden_error = (weight_k * error_j) * transfer_derivative(output)
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
            print("Error : " + str(neuron['delta']))

In [9]:
# Error Backpropagation
backward_propagate_error(network, expected)

Error : -0.13849856162855698
Error : 0.03809823651655623
Error : -0.008771354689486937
Error : -0.009954254705217202


### 4. Update and Train Network

This part is broken down into two sections:
- Update Weights.
- Train Network.

In [10]:
# Update network weights with error
# weight = weight + learning_rate * error * input
# weight = weight + learning_rate * error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)): 
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
				print(neuron['delta'], neuron['weights'][j])                
			neuron['weights'][-1] += l_rate * neuron['delta']
			print(neuron['delta'], neuron['weights'][-1])

In [11]:
l_rate = 0.9
print("Error \t\t\t New weight")
update_weights(network, row_data, l_rate)

Error 			 New weight
-0.008771354689486937 0.14960528903897308
-0.008771354689486937 0.19921057807794618
-0.008771354689486937 0.34210578077946174
-0.009954254705217202 0.24955205853826523
-0.009954254705217202 0.29910411707653045
-0.009954254705217202 0.3410411707653045
-0.13849856162855698 0.32604966349219233
-0.13849856162855698 0.3755991349372201
-0.13849856162855698 0.4753512945342987
0.03809823651655623 0.5203422864297276
0.03809823651655623 0.5704662179943805
0.03809823651655623 0.6342884128649006


In [12]:
network

[[{'weights': [0.14960528903897308, 0.19921057807794618, 0.34210578077946174],
   'output': 0.5932699921071872,
   'delta': -0.008771354689486937},
  {'weights': [0.24955205853826523, 0.29910411707653045, 0.3410411707653045],
   'output': 0.596884378259767,
   'delta': -0.009954254705217202}],
 [{'weights': [0.32604966349219233, 0.3755991349372201, 0.4753512945342987],
   'output': 0.7513650695523157,
   'delta': -0.13849856162855698},
  {'weights': [0.5203422864297276, 0.5704662179943805, 0.6342884128649006],
   'output': 0.7729284653214625,
   'delta': 0.03809823651655623}]]

In [13]:
# Train a network for a fixed number of epochs (iterations)
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
		print('========================')

In [14]:
# Initialize a network function - init random weight and bias values
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [15]:
# Training a network with 2 inputs, 2 neurons hidden layer, and 2 neuron output layer
from random import seed
from random import random
seed(1)
train_dataset = [[2.7810836,2.550537003,0],
            [1.465489372,2.362125076,0],
            [3.396561688,4.400293529,0],
            [8.675418651,-0.242068655,1],
            [7.673756466,3.508563011,1]]
n_inputs = len(train_dataset[0]) - 1
n_outputs = len(set([row[-1] for row in train_dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
n_epoch = 2
l_rate = 0.5
train_network(network, train_dataset, l_rate, n_epoch, n_outputs)

Inputs : 3.2988639183529895
Outputs : 0.9643898158763548
Inputs : 2.422484871353145
Outputs : 0.9185258960543243
Inputs : 1.4467120366053836
Outputs : 0.8094918973879515
Inputs : 1.227776976537604
Outputs : 0.7734292563511262
Error : 0.029379162360871295
Error : -0.13553299084477086
Error : 0.000525476610401129
Error : -0.0067428584755714034
0.000525476610401129 0.1350949413040863
0.000525476610401129 0.8481038607067523
0.000525476610401129 0.7640373572818145
-0.0067428584755714034 0.24569279917765538
-0.0067428584755714034 0.48683613206797244
-0.0067428584755714034 0.4461196355509524
0.029379162360871295 0.665759455212664
0.029379162360871295 0.8022161118519355
0.029379162360871295 0.10854916795467054
-0.13553299084477086 -0.03700584152097382
-0.13553299084477086 0.7735198229895619
-0.13553299084477086 0.36500057248266793
Inputs : 2.9653449544017474
Outputs : 0.9509837432440319
Inputs : 1.956147656983342
Outputs : 0.8761154354578115
Inputs : 1.4445095049393482
Outputs : 0.809152002926

In [16]:
network

[[{'weights': [-0.016165803091048196, 0.8537471385068439, 0.7486799239758004],
   'output': 0.9760874069494646,
   'delta': -0.0024680899690765158},
  {'weights': [0.16466049914824785, 0.4428613247080777, 0.42553222860857204],
   'output': 0.9641966421973981,
   'delta': -0.001394801100982609}],
 [{'weights': [0.5138698169765883, 0.6363266183567993, -0.06691491685638438],
   'output': 0.7760172690308134,
   'delta': -0.13488302815208458},
  {'weights': [-0.23497093478527004, 0.60285524533664, 0.1744803648458224],
   'output': 0.595289832931719,
   'delta': 0.09750271182885753}]]

### 5. Predict

In [17]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

In [18]:
# Test making predictions with the network
test_dataset = [[2.7810836,2.550537003,0],
                [1.465489372,2.362125076,0],
                [7.627531214,2.759262235,1],
                [5.332441248,2.088626775,1]]

for row in test_dataset:
    prediction = predict(network, row)
    print('\nExpected=%d, Got=%d' % (row[-1], prediction))
    print("===============")

Inputs : 2.8812351420855284
Outputs : 0.9469109891811595
Inputs : 2.0130010382231287
Outputs : 0.8821553599383366
Inputs : 0.981012996902145
Outputs : 0.7273091708199171
Inputs : 0.48379579050013855
Outputs : 0.6186437939180887

Expected=0, Got=0
Inputs : 2.741646635786285
Outputs : 0.939439845926922
Inputs : 1.7129342803820728
Outputs : 0.8472164866640284
Inputs : 0.9549412668056468
Outputs : 0.7221078201236351
Inputs : 0.4644882089949326
Outputs : 0.6140783624921721

Expected=0, Got=0
Inputs : 2.9810869938207016
Outputs : 0.951712349652103
Inputs : 2.903455854183724
Outputs : 0.9480170079664101
Inputs : 1.0253897908976972
Outputs : 0.7360211349362572
Inputs : 0.5223726503222796
Outputs : 0.6277024037314222

Expected=1, Got=0
Inputs : 2.445635861331077
Outputs : 0.920241723478603
Inputs : 2.2285466865802177
Outputs : 0.9027838833959835
Inputs : 0.9804349447901706
Outputs : 0.7271945103691089
Inputs : 0.5024983063622974
Outputs : 0.6230462625995964

Expected=1, Got=0
